# 6t

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (although only 6H will be used)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))

# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (only using 6H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        six_hour = convert_to_one_hot(sample['6H'], self.total_variables)
        return six_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)  # (batch_size, seq_len, input_size) -> (batch_size, seq_len, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)  # dropout=0.0으로 설정
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, seq_len, hidden_size)
        x = x.transpose(0, 1)  # Shape: (seq_len, batch_size, hidden_size)
        x = self.transformer_encoder(x)  # Shape: (seq_len, batch_size, hidden_size)
        x = x.transpose(0, 1)  # Shape: (batch_size, seq_len, hidden_size)
        x = x[:, -1, :]
        #x = x.mean(dim=1)  # Shape: (batch_size, hidden_size)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, six_hour):
        transformer_out = self.transformer(six_hour)  # Shape: (batch_size, hidden_size)
        output = self.mlp(transformer_out)  # Shape: (batch_size, 1)
        return output



# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for six_hour, labels in val_loader:
            six_hour, labels = six_hour.to(device), labels.to(device)
            outputs = model(six_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []  # To store metrics for each epoch

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for six_hour, labels in train_loader:
            six_hour, labels = six_hour.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(six_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        # Log the metrics for this epoch
        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,  # Save current fold number
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch  # Track the best epoch
        }

        # Add this epoch's result to the list
        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    # Save the best model
    if best_model_state is not None:
        model_name = f'best_transformer_6H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel2(icd_10)/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Set the seed for reproducibility
# seed = 42  # You can choose any seed value
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize lists to store results for all folds
all_fold_results = []

# Start tracking time for the 6H training
start_time = time.time()

# Loop through the folds
fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    # Prepare data loaders for this fold (only using 6H data)
    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    # Initialize model
    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train the model and capture results for this fold
    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    # Append the results for this fold to all_fold_results
    all_fold_results.extend(fold_results)

    fold += 1

# After all folds are done, save the results to a single CSV file
results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_6H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata3(icd_10)/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

# Stop the timer for the 6H training
end_time = time.time()
total_time = end_time - start_time

# Display the total time taken for the training
print(f"Time taken for 6H training: {total_time:.2f} seconds")

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 1 - Train Loss: 0.7133, Val Loss: 0.6906, Train AUC-ROC: 0.5279, Val AUC-ROC: 0.6303, Train Accuracy: 0.5269, Val Accuracy: 0.5520
Epoch 2/100, Fold 1 - Train Loss: 0.6910, Val Loss: 0.6930, Train AUC-ROC: 0.6107, Val AUC-ROC: 0.6851, Train Accuracy: 0.5570, Val Accuracy: 0.5394
Epoch 3/100, Fold 1 - Train Loss: 0.6714, Val Loss: 0.6939, Train AUC-ROC: 0.6879, Val AUC-ROC: 0.7248, Train Accuracy: 0.5975, Val Accuracy: 0.5217
Epoch 4/100, Fold 1 - Train Loss: 0.6568, Val Loss: 0.6920, Train AUC-ROC: 0.7270, Val AUC-ROC: 0.7528, Train Accuracy: 0.6160, Val Accuracy: 0.5166
Epoch 5/100, Fold 1 - Train Loss: 0.6438, Val Loss: 0.6868, Train AUC-ROC: 0.7520, Val AUC-ROC: 0.7722, Train Accuracy: 0.6355, Val Accuracy: 0.5483
Epoch 6/100, Fold 1 - Train Loss: 0.6297, Val Loss: 0.6782, Train AUC-ROC: 0.7824, Val AUC-ROC: 0.7861, Train Accuracy: 0.6576, Val Accuracy: 0.5829
Epoch 7/100, Fold 1 - Train Loss: 0.6228, Val Loss: 0.6655, Train AUC-ROC: 0.8028, Val AUC-ROC: 0.7950, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 2 - Train Loss: 0.6861, Val Loss: 0.6610, Train AUC-ROC: 0.4343, Val AUC-ROC: 0.4998, Train Accuracy: 0.5664, Val Accuracy: 0.6750
Epoch 2/100, Fold 2 - Train Loss: 0.6577, Val Loss: 0.6563, Train AUC-ROC: 0.5292, Val AUC-ROC: 0.6241, Train Accuracy: 0.6086, Val Accuracy: 0.6971
Epoch 3/100, Fold 2 - Train Loss: 0.6376, Val Loss: 0.6508, Train AUC-ROC: 0.6023, Val AUC-ROC: 0.6992, Train Accuracy: 0.6434, Val Accuracy: 0.6912
Epoch 4/100, Fold 2 - Train Loss: 0.6156, Val Loss: 0.6437, Train AUC-ROC: 0.6910, Val AUC-ROC: 0.7398, Train Accuracy: 0.6775, Val Accuracy: 0.6824
Epoch 5/100, Fold 2 - Train Loss: 0.6063, Val Loss: 0.6349, Train AUC-ROC: 0.7154, Val AUC-ROC: 0.7617, Train Accuracy: 0.6954, Val Accuracy: 0.6986
Epoch 6/100, Fold 2 - Train Loss: 0.5918, Val Loss: 0.6238, Train AUC-ROC: 0.7602, Val AUC-ROC: 0.7756, Train Accuracy: 0.7109, Val Accuracy: 0.7155
Epoch 7/100, Fold 2 - Train Loss: 0.5831, Val Loss: 0.6103, Train AUC-ROC: 0.7864, Val AUC-ROC: 0.7858, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 1/100, Fold 8 - Train Loss: 0.6656, Val Loss: 0.7392, Train AUC-ROC: 0.5086, Val AUC-ROC: 0.6371, Train Accuracy: 0.5985, Val Accuracy: 0.2970
Epoch 2/100, Fold 8 - Train Loss: 0.6477, Val Loss: 0.7299, Train AUC-ROC: 0.5767, Val AUC-ROC: 0.7155, Train Accuracy: 0.6184, Val Accuracy: 0.3441
Epoch 3/100, Fold 8 - Train Loss: 0.6298, Val Loss: 0.7182, Train AUC-ROC: 0.6521, Val AUC-ROC: 0.7592, Train Accuracy: 0.6423, Val Accuracy: 0.3994
Epoch 4/100, Fold 8 - Train Loss: 0.6100, Val Loss: 0.7040, Train AUC-ROC: 0.7094, Val AUC-ROC: 0.7828, Train Accuracy: 0.6744, Val Accuracy: 0.4687
Epoch 5/100, Fold 8 - Train Loss: 0.5936, Val Loss: 0.6870, Train AUC-ROC: 0.7611, Val AUC-ROC: 0.7976, Train Accuracy: 0.6952, Val Accuracy: 0.5380
Epoch 6/100, Fold 8 - Train Loss: 0.5893, Val Loss: 0.6675, Train AUC-ROC: 0.7705, Val AUC-ROC: 0.8075, Train Accuracy: 0.6976, Val Accuracy: 0.5903
Epoch 7/100, Fold 8 - Train Loss: 0.5782, Val Loss: 0.6457, Train AUC-ROC: 0.7946, Val AUC-ROC: 0.8148, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 9 - Train Loss: 0.6996, Val Loss: 0.7081, Train AUC-ROC: 0.5012, Val AUC-ROC: 0.6215, Train Accuracy: 0.5514, Val Accuracy: 0.4591
Epoch 2/100, Fold 9 - Train Loss: 0.6689, Val Loss: 0.6989, Train AUC-ROC: 0.5955, Val AUC-ROC: 0.7123, Train Accuracy: 0.5868, Val Accuracy: 0.4827
Epoch 3/100, Fold 9 - Train Loss: 0.6478, Val Loss: 0.6865, Train AUC-ROC: 0.6715, Val AUC-ROC: 0.7617, Train Accuracy: 0.6174, Val Accuracy: 0.5306
Epoch 4/100, Fold 9 - Train Loss: 0.6310, Val Loss: 0.6709, Train AUC-ROC: 0.7173, Val AUC-ROC: 0.7883, Train Accuracy: 0.6318, Val Accuracy: 0.5829
Epoch 5/100, Fold 9 - Train Loss: 0.6217, Val Loss: 0.6522, Train AUC-ROC: 0.7460, Val AUC-ROC: 0.8043, Train Accuracy: 0.6541, Val Accuracy: 0.6271
Epoch 6/100, Fold 9 - Train Loss: 0.6064, Val Loss: 0.6303, Train AUC-ROC: 0.7832, Val AUC-ROC: 0.8149, Train Accuracy: 0.6715, Val Accuracy: 0.6647
Epoch 7/100, Fold 9 - Train Loss: 0.5992, Val Loss: 0.6051, Train AUC-ROC: 0.7969, Val AUC-ROC: 0.8221, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 10 - Train Loss: 0.5372, Val Loss: 0.5891, Train AUC-ROC: 0.4939, Val AUC-ROC: 0.5612, Train Accuracy: 0.7978, Val Accuracy: 0.8209
Epoch 2/100, Fold 10 - Train Loss: 0.5125, Val Loss: 0.5800, Train AUC-ROC: 0.5791, Val AUC-ROC: 0.6532, Train Accuracy: 0.7976, Val Accuracy: 0.8202
Epoch 3/100, Fold 10 - Train Loss: 0.4886, Val Loss: 0.5710, Train AUC-ROC: 0.6634, Val AUC-ROC: 0.7113, Train Accuracy: 0.8110, Val Accuracy: 0.8290
Epoch 4/100, Fold 10 - Train Loss: 0.4778, Val Loss: 0.5614, Train AUC-ROC: 0.7054, Val AUC-ROC: 0.7472, Train Accuracy: 0.8118, Val Accuracy: 0.8475
Epoch 5/100, Fold 10 - Train Loss: 0.4638, Val Loss: 0.5510, Train AUC-ROC: 0.7515, Val AUC-ROC: 0.7714, Train Accuracy: 0.8265, Val Accuracy: 0.8504
Epoch 6/100, Fold 10 - Train Loss: 0.4506, Val Loss: 0.5392, Train AUC-ROC: 0.7853, Val AUC-ROC: 0.7897, Train Accuracy: 0.8317, Val Accuracy: 0.8526
Epoch 7/100, Fold 10 - Train Loss: 0.4446, Val Loss: 0.5254, Train AUC-ROC: 0.7998, Val AUC-ROC: 0.8

# 12t

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (although only 12H will be used)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))

# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (only using 12H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        twelve_hour = convert_to_one_hot(sample['12H'], self.total_variables)
        return twelve_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)  # (batch_size, seq_len, input_size) -> (batch_size, seq_len, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)  # dropout=0.0으로 설정
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, seq_len, hidden_size)
        x = x.transpose(0, 1)  # Shape: (seq_len, batch_size, hidden_size)
        x = self.transformer_encoder(x)  # Shape: (seq_len, batch_size, hidden_size)
        x = x.transpose(0, 1)  # Shape: (batch_size, seq_len, hidden_size)
        x = x[:, -1, :]
        #x = x.mean(dim=1)  # Shape: (batch_size, hidden_size)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  # MLP에서만 드롭아웃을 유지
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, twelve_hour):
        transformer_out = self.transformer(twelve_hour)  # Shape: (batch_size, hidden_size)
        output = self.mlp(transformer_out)  # Shape: (batch_size, 1)
        return output

# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for twelve_hour, labels in val_loader:
            twelve_hour, labels = twelve_hour.to(device), labels.to(device)
            outputs = model(twelve_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []  # To store metrics for each epoch

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for twelve_hour, labels in train_loader:
            twelve_hour, labels = twelve_hour.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(twelve_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        # Log the metrics for this epoch
        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,  # Save current fold number
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch  # Track the best epoch
        }

        # Add this epoch's result to the list
        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    # Save the best model
    if best_model_state is not None:
        model_name = f'best_transformer_12H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel3(icd_10)/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize lists to store results for all folds
all_fold_results = []

# Start tracking time for the 12H training
start_time = time.time()

# Loop through the folds
fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    # Prepare data loaders for this fold (only using 12H data)
    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    # Initialize model
    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train the model and capture results for this fold
    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    # Append the results for this fold to all_fold_results
    all_fold_results.extend(fold_results)

    fold += 1

# After all folds are done, save the results to a single CSV file
results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_12H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata3(icd_10)/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

# Stop the timer for the 12H training
end_time = time.time()
total_time = end_time - start_time

# Display the total time taken for the training
print(f"Time taken for 12H training: {total_time:.2f} seconds")


/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 1 - Train Loss: 0.6744, Val Loss: 0.6806, Train AUC-ROC: 0.4818, Val AUC-ROC: 0.6265, Train Accuracy: 0.5843, Val Accuracy: 0.5770
Epoch 2/100, Fold 1 - Train Loss: 0.6384, Val Loss: 0.6799, Train AUC-ROC: 0.5998, Val AUC-ROC: 0.7163, Train Accuracy: 0.6379, Val Accuracy: 0.5689
Epoch 3/100, Fold 1 - Train Loss: 0.6192, Val Loss: 0.6766, Train AUC-ROC: 0.6751, Val AUC-ROC: 0.7525, Train Accuracy: 0.6611, Val Accuracy: 0.5615
Epoch 4/100, Fold 1 - Train Loss: 0.6036, Val Loss: 0.6704, Train AUC-ROC: 0.7234, Val AUC-ROC: 0.7692, Train Accuracy: 0.6775, Val Accuracy: 0.5829
Epoch 5/100, Fold 1 - Train Loss: 0.5885, Val Loss: 0.6606, Train AUC-ROC: 0.7707, Val AUC-ROC: 0.7808, Train Accuracy: 0.6917, Val Accuracy: 0.6072
Epoch 6/100, Fold 1 - Train Loss: 0.5828, Val Loss: 0.6470, Train AUC-ROC: 0.7865, Val AUC-ROC: 0.7901, Train Accuracy: 0.7052, Val Accuracy: 0.6293
Epoch 7/100, Fold 1 - Train Loss: 0.5757, Val Loss: 0.6294, Train AUC-ROC: 0.8034, Val AUC-ROC: 0.7977, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 2 - Train Loss: 0.8719, Val Loss: 0.8175, Train AUC-ROC: 0.4976, Val AUC-ROC: 0.6187, Train Accuracy: 0.3271, Val Accuracy: 0.1857
Epoch 2/100, Fold 2 - Train Loss: 0.8460, Val Loss: 0.8190, Train AUC-ROC: 0.5884, Val AUC-ROC: 0.6892, Train Accuracy: 0.3662, Val Accuracy: 0.1982
Epoch 3/100, Fold 2 - Train Loss: 0.8255, Val Loss: 0.8181, Train AUC-ROC: 0.6437, Val AUC-ROC: 0.7312, Train Accuracy: 0.3883, Val Accuracy: 0.2181
Epoch 4/100, Fold 2 - Train Loss: 0.8108, Val Loss: 0.8129, Train AUC-ROC: 0.7109, Val AUC-ROC: 0.7565, Train Accuracy: 0.4169, Val Accuracy: 0.2447
Epoch 5/100, Fold 2 - Train Loss: 0.8009, Val Loss: 0.8019, Train AUC-ROC: 0.7228, Val AUC-ROC: 0.7730, Train Accuracy: 0.4428, Val Accuracy: 0.2881
Epoch 6/100, Fold 2 - Train Loss: 0.7909, Val Loss: 0.7867, Train AUC-ROC: 0.7477, Val AUC-ROC: 0.7840, Train Accuracy: 0.4556, Val Accuracy: 0.3338
Epoch 7/100, Fold 2 - Train Loss: 0.7766, Val Loss: 0.7670, Train AUC-ROC: 0.7748, Val AUC-ROC: 0.7924, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 1/100, Fold 8 - Train Loss: 0.7252, Val Loss: 0.6942, Train AUC-ROC: 0.4461, Val AUC-ROC: 0.6006, Train Accuracy: 0.4930, Val Accuracy: 0.4974
Epoch 2/100, Fold 8 - Train Loss: 0.6864, Val Loss: 0.6958, Train AUC-ROC: 0.5711, Val AUC-ROC: 0.7012, Train Accuracy: 0.5627, Val Accuracy: 0.4974
Epoch 3/100, Fold 8 - Train Loss: 0.6731, Val Loss: 0.6957, Train AUC-ROC: 0.6497, Val AUC-ROC: 0.7539, Train Accuracy: 0.5758, Val Accuracy: 0.5011
Epoch 4/100, Fold 8 - Train Loss: 0.6484, Val Loss: 0.6933, Train AUC-ROC: 0.7254, Val AUC-ROC: 0.7817, Train Accuracy: 0.6228, Val Accuracy: 0.5166
Epoch 5/100, Fold 8 - Train Loss: 0.6387, Val Loss: 0.6874, Train AUC-ROC: 0.7481, Val AUC-ROC: 0.7971, Train Accuracy: 0.6296, Val Accuracy: 0.5387
Epoch 6/100, Fold 8 - Train Loss: 0.6313, Val Loss: 0.6780, Train AUC-ROC: 0.7737, Val AUC-ROC: 0.8064, Train Accuracy: 0.6388, Val Accuracy: 0.5660
Epoch 7/100, Fold 8 - Train Loss: 0.6272, Val Loss: 0.6647, Train AUC-ROC: 0.7867, Val AUC-ROC: 0.8122, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 9 - Train Loss: 0.6205, Val Loss: 0.6663, Train AUC-ROC: 0.5788, Val AUC-ROC: 0.7488, Train Accuracy: 0.6656, Val Accuracy: 0.6360
Epoch 2/100, Fold 9 - Train Loss: 0.5944, Val Loss: 0.6572, Train AUC-ROC: 0.6783, Val AUC-ROC: 0.7904, Train Accuracy: 0.7070, Val Accuracy: 0.6500
Epoch 3/100, Fold 9 - Train Loss: 0.5787, Val Loss: 0.6449, Train AUC-ROC: 0.7256, Val AUC-ROC: 0.8083, Train Accuracy: 0.7183, Val Accuracy: 0.6676
Epoch 4/100, Fold 9 - Train Loss: 0.5603, Val Loss: 0.6280, Train AUC-ROC: 0.7739, Val AUC-ROC: 0.8165, Train Accuracy: 0.7336, Val Accuracy: 0.6949
Epoch 5/100, Fold 9 - Train Loss: 0.5543, Val Loss: 0.6075, Train AUC-ROC: 0.7877, Val AUC-ROC: 0.8206, Train Accuracy: 0.7371, Val Accuracy: 0.7303
Epoch 6/100, Fold 9 - Train Loss: 0.5419, Val Loss: 0.5840, Train AUC-ROC: 0.8058, Val AUC-ROC: 0.8227, Train Accuracy: 0.7528, Val Accuracy: 0.7701
Epoch 7/100, Fold 9 - Train Loss: 0.5315, Val Loss: 0.5588, Train AUC-ROC: 0.8275, Val AUC-ROC: 0.8239, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 10 - Train Loss: 0.7169, Val Loss: 0.6687, Train AUC-ROC: 0.4078, Val AUC-ROC: 0.5248, Train Accuracy: 0.5074, Val Accuracy: 0.6463
Epoch 2/100, Fold 10 - Train Loss: 0.6863, Val Loss: 0.6665, Train AUC-ROC: 0.4999, Val AUC-ROC: 0.6669, Train Accuracy: 0.5621, Val Accuracy: 0.6529
Epoch 3/100, Fold 10 - Train Loss: 0.6592, Val Loss: 0.6641, Train AUC-ROC: 0.5919, Val AUC-ROC: 0.7489, Train Accuracy: 0.6040, Val Accuracy: 0.6441
Epoch 4/100, Fold 10 - Train Loss: 0.6440, Val Loss: 0.6607, Train AUC-ROC: 0.6657, Val AUC-ROC: 0.7881, Train Accuracy: 0.6309, Val Accuracy: 0.6345
Epoch 5/100, Fold 10 - Train Loss: 0.6274, Val Loss: 0.6557, Train AUC-ROC: 0.7132, Val AUC-ROC: 0.8091, Train Accuracy: 0.6482, Val Accuracy: 0.6374
Epoch 6/100, Fold 10 - Train Loss: 0.6175, Val Loss: 0.6480, Train AUC-ROC: 0.7410, Val AUC-ROC: 0.8215, Train Accuracy: 0.6593, Val Accuracy: 0.6470
Epoch 7/100, Fold 10 - Train Loss: 0.6058, Val Loss: 0.6368, Train AUC-ROC: 0.7663, Val AUC-ROC: 0.8

# 24t

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (although only 24H will be used)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))

# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (only using 24H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        twenty_four_hour = convert_to_one_hot(sample['24H'], self.total_variables)
        return twenty_four_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)  # (batch_size, seq_len, input_size) -> (batch_size, seq_len, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)  # dropout=0.0으로 설정
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, seq_len, hidden_size)
        x = x.transpose(0, 1)  # Shape: (seq_len, batch_size, hidden_size)
        x = self.transformer_encoder(x)  # Shape: (seq_len, batch_size, hidden_size)
        x = x.transpose(0, 1)  # Shape: (batch_size, seq_len, hidden_size)
        x = x[:, -1, :]
        #x = x.mean(dim=1)  # Shape: (batch_size, hidden_size)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  # MLP에서만 드롭아웃을 유지
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, twenty_four_hour):
        transformer_out = self.transformer(twenty_four_hour)  # Shape: (batch_size, hidden_size)
        output = self.mlp(transformer_out)  # Shape: (batch_size, 1)
        return output

# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for twenty_four_hour, labels in val_loader:
            twenty_four_hour, labels = twenty_four_hour.to(device), labels.to(device)
            outputs = model(twenty_four_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []  # To store metrics for each epoch

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for twenty_four_hour, labels in train_loader:
            twenty_four_hour, labels = twenty_four_hour.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(twenty_four_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        # Log the metrics for this epoch
        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,  # Save current fold number
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch  # Track the best epoch
        }

        # Add this epoch's result to the list
        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    # Save the best model
    if best_model_state is not None:
        model_name = f'best_transformer_24H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel3(icd_10)/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Set the seed for reproducibility
# seed = 42  # You can choose any seed value
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize lists to store results for all folds
all_fold_results = []

# Start tracking time for the 24H training
start_time = time.time()

# Loop through the folds
fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    # Prepare data loaders for this fold (only using 24H data)
    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    # Initialize model
    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train the model and capture results for this fold
    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    # Append the results for this fold to all_fold_results
    all_fold_results.extend(fold_results)

    fold += 1

# After all folds are done, save the results to a single CSV file
results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_24H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata3(icd_10)/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

# Stop the timer for the 24H training
end_time = time.time()
total_time = end_time - start_time

# Display the total time taken for the training
print(f"Time taken for 24H training: {total_time:.2f} seconds")


/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 1 - Train Loss: 0.6839, Val Loss: 0.7377, Train AUC-ROC: 0.5468, Val AUC-ROC: 0.7020, Train Accuracy: 0.5669, Val Accuracy: 0.3412
Epoch 2/100, Fold 1 - Train Loss: 0.6623, Val Loss: 0.7281, Train AUC-ROC: 0.6356, Val AUC-ROC: 0.7318, Train Accuracy: 0.6064, Val Accuracy: 0.4141
Epoch 3/100, Fold 1 - Train Loss: 0.6460, Val Loss: 0.7157, Train AUC-ROC: 0.6926, Val AUC-ROC: 0.7475, Train Accuracy: 0.6329, Val Accuracy: 0.4613
Epoch 4/100, Fold 1 - Train Loss: 0.6361, Val Loss: 0.7009, Train AUC-ROC: 0.7224, Val AUC-ROC: 0.7584, Train Accuracy: 0.6499, Val Accuracy: 0.5151
Epoch 5/100, Fold 1 - Train Loss: 0.6241, Val Loss: 0.6833, Train AUC-ROC: 0.7492, Val AUC-ROC: 0.7685, Train Accuracy: 0.6611, Val Accuracy: 0.5586
Epoch 6/100, Fold 1 - Train Loss: 0.6158, Val Loss: 0.6639, Train AUC-ROC: 0.7692, Val AUC-ROC: 0.7777, Train Accuracy: 0.6716, Val Accuracy: 0.6065
Epoch 7/100, Fold 1 - Train Loss: 0.6011, Val Loss: 0.6428, Train AUC-ROC: 0.7865, Val AUC-ROC: 0.7859, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 2 - Train Loss: 0.8008, Val Loss: 0.7794, Train AUC-ROC: 0.4677, Val AUC-ROC: 0.5403, Train Accuracy: 0.3992, Val Accuracy: 0.2034
Epoch 2/100, Fold 2 - Train Loss: 0.7700, Val Loss: 0.7743, Train AUC-ROC: 0.5692, Val AUC-ROC: 0.6533, Train Accuracy: 0.4377, Val Accuracy: 0.2395
Epoch 3/100, Fold 2 - Train Loss: 0.7547, Val Loss: 0.7685, Train AUC-ROC: 0.6320, Val AUC-ROC: 0.7155, Train Accuracy: 0.4747, Val Accuracy: 0.2786
Epoch 4/100, Fold 2 - Train Loss: 0.7394, Val Loss: 0.7617, Train AUC-ROC: 0.6776, Val AUC-ROC: 0.7493, Train Accuracy: 0.5000, Val Accuracy: 0.3125
Epoch 5/100, Fold 2 - Train Loss: 0.7219, Val Loss: 0.7531, Train AUC-ROC: 0.7252, Val AUC-ROC: 0.7672, Train Accuracy: 0.5308, Val Accuracy: 0.3500
Epoch 6/100, Fold 2 - Train Loss: 0.7132, Val Loss: 0.7420, Train AUC-ROC: 0.7497, Val AUC-ROC: 0.7776, Train Accuracy: 0.5326, Val Accuracy: 0.3861
Epoch 7/100, Fold 2 - Train Loss: 0.7052, Val Loss: 0.7284, Train AUC-ROC: 0.7600, Val AUC-ROC: 0.7831, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 1/100, Fold 8 - Train Loss: 0.7494, Val Loss: 0.7755, Train AUC-ROC: 0.5317, Val AUC-ROC: 0.6057, Train Accuracy: 0.4897, Val Accuracy: 0.2041
Epoch 2/100, Fold 8 - Train Loss: 0.7122, Val Loss: 0.7711, Train AUC-ROC: 0.6376, Val AUC-ROC: 0.6754, Train Accuracy: 0.5415, Val Accuracy: 0.2373
Epoch 3/100, Fold 8 - Train Loss: 0.6997, Val Loss: 0.7628, Train AUC-ROC: 0.6868, Val AUC-ROC: 0.7138, Train Accuracy: 0.5527, Val Accuracy: 0.2771
Epoch 4/100, Fold 8 - Train Loss: 0.6812, Val Loss: 0.7511, Train AUC-ROC: 0.7228, Val AUC-ROC: 0.7370, Train Accuracy: 0.5854, Val Accuracy: 0.3294
Epoch 5/100, Fold 8 - Train Loss: 0.6707, Val Loss: 0.7353, Train AUC-ROC: 0.7529, Val AUC-ROC: 0.7523, Train Accuracy: 0.6036, Val Accuracy: 0.4024
Epoch 6/100, Fold 8 - Train Loss: 0.6654, Val Loss: 0.7152, Train AUC-ROC: 0.7644, Val AUC-ROC: 0.7631, Train Accuracy: 0.6095, Val Accuracy: 0.4709
Epoch 7/100, Fold 8 - Train Loss: 0.6480, Val Loss: 0.6909, Train AUC-ROC: 0.7901, Val AUC-ROC: 0.7707, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 9 - Train Loss: 0.7405, Val Loss: 0.6539, Train AUC-ROC: 0.4450, Val AUC-ROC: 0.5667, Train Accuracy: 0.4770, Val Accuracy: 0.6993
Epoch 2/100, Fold 9 - Train Loss: 0.7044, Val Loss: 0.6578, Train AUC-ROC: 0.5684, Val AUC-ROC: 0.6391, Train Accuracy: 0.5367, Val Accuracy: 0.6293
Epoch 3/100, Fold 9 - Train Loss: 0.6866, Val Loss: 0.6607, Train AUC-ROC: 0.6445, Val AUC-ROC: 0.6838, Train Accuracy: 0.5658, Val Accuracy: 0.5940
Epoch 4/100, Fold 9 - Train Loss: 0.6731, Val Loss: 0.6619, Train AUC-ROC: 0.6852, Val AUC-ROC: 0.7179, Train Accuracy: 0.5804, Val Accuracy: 0.5866
Epoch 5/100, Fold 9 - Train Loss: 0.6595, Val Loss: 0.6608, Train AUC-ROC: 0.7240, Val AUC-ROC: 0.7431, Train Accuracy: 0.6047, Val Accuracy: 0.5917
Epoch 6/100, Fold 9 - Train Loss: 0.6546, Val Loss: 0.6569, Train AUC-ROC: 0.7448, Val AUC-ROC: 0.7630, Train Accuracy: 0.6018, Val Accuracy: 0.6057
Epoch 7/100, Fold 9 - Train Loss: 0.6478, Val Loss: 0.6494, Train AUC-ROC: 0.7571, Val AUC-ROC: 0.7780, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 10 - Train Loss: 0.7595, Val Loss: 0.6862, Train AUC-ROC: 0.4754, Val AUC-ROC: 0.5543, Train Accuracy: 0.4801, Val Accuracy: 0.5844
Epoch 2/100, Fold 10 - Train Loss: 0.7302, Val Loss: 0.6901, Train AUC-ROC: 0.5676, Val AUC-ROC: 0.6576, Train Accuracy: 0.5186, Val Accuracy: 0.5578
Epoch 3/100, Fold 10 - Train Loss: 0.7056, Val Loss: 0.6922, Train AUC-ROC: 0.6411, Val AUC-ROC: 0.7115, Train Accuracy: 0.5546, Val Accuracy: 0.5321
Epoch 4/100, Fold 10 - Train Loss: 0.6881, Val Loss: 0.6914, Train AUC-ROC: 0.6879, Val AUC-ROC: 0.7429, Train Accuracy: 0.5713, Val Accuracy: 0.5254
Epoch 5/100, Fold 10 - Train Loss: 0.6738, Val Loss: 0.6879, Train AUC-ROC: 0.7249, Val AUC-ROC: 0.7632, Train Accuracy: 0.5951, Val Accuracy: 0.5387
Epoch 6/100, Fold 10 - Train Loss: 0.6648, Val Loss: 0.6809, Train AUC-ROC: 0.7473, Val AUC-ROC: 0.7769, Train Accuracy: 0.6121, Val Accuracy: 0.5623
Epoch 7/100, Fold 10 - Train Loss: 0.6498, Val Loss: 0.6710, Train AUC-ROC: 0.7674, Val AUC-ROC: 0.7

# 48t

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (although only 48H will be used)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))


# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (only using 48H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        forty_eight_hour = convert_to_one_hot(sample['48H'], self.total_variables)
        return forty_eight_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)  # (batch_size, seq_len, input_size) -> (batch_size, seq_len, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)  # dropout=0.0으로 설정
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)  # Shape: (batch_size, seq_len, hidden_size)
        x = x.transpose(0, 1)  # Shape: (seq_len, batch_size, hidden_size)
        x = self.transformer_encoder(x)  # Shape: (seq_len, batch_size, hidden_size)
        x = x.transpose(0, 1)  # Shape: (batch_size, seq_len, hidden_size)
        x = x[:, -1, :]
        #x = x.mean(dim=1)  # Shape: (batch_size, hidden_size)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  # MLP에서만 드롭아웃을 유지

            #nn.Linear(mlp_hidden_size, mlp_hidden_size),
            #nn.BatchNorm1d(mlp_hidden_size),
            #nn.ReLU(),
            #nn.Dropout(dropout_prob),
            
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, forty_eight_hour):
        transformer_out = self.transformer(forty_eight_hour)  # Shape: (batch_size, hidden_size)
        output = self.mlp(transformer_out)  # Shape: (batch_size, 1)
        return output

# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for forty_eight_hour, labels in val_loader:
            forty_eight_hour, labels = forty_eight_hour.to(device), labels.to(device)
            outputs = model(forty_eight_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []  # To store metrics for each epoch

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for forty_eight_hour, labels in train_loader:
            forty_eight_hour, labels = forty_eight_hour.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(forty_eight_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        # Log the metrics for this epoch
        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,  # Save current fold number
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch  # Track the best epoch
        }

        # Add this epoch's result to the list
        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    # Save the best model
    if best_model_state is not None:
        model_name = f'best_transformer_48H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel3(icd_10)/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Set the seed for reproducibility
# seed = 42  # You can choose any seed value
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize lists to store results for all folds
all_fold_results = []

# Start tracking time for the 48H training
start_time = time.time()

# Loop through the folds
fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    # Prepare data loaders for this fold (only using 48H data)
    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    # Initialize model
    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train the model and capture results for this fold
    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    # Append the results for this fold to all_fold_results
    all_fold_results.extend(fold_results)

    fold += 1

# After all folds are done, save the results to a single CSV file
results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_48H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata3(icd_10)/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

# Stop the timer for the 48H training
end_time = time.time()
total_time = end_time - start_time

# Display the total time taken for the training
print(f"Time taken for 48H training: {total_time:.2f} seconds")

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 1 - Train Loss: 0.6752, Val Loss: 0.6722, Train AUC-ROC: 0.4934, Val AUC-ROC: 0.6077, Train Accuracy: 0.5808, Val Accuracy: 0.6551
Epoch 2/100, Fold 1 - Train Loss: 0.6459, Val Loss: 0.6726, Train AUC-ROC: 0.6177, Val AUC-ROC: 0.6764, Train Accuracy: 0.6405, Val Accuracy: 0.6227
Epoch 3/100, Fold 1 - Train Loss: 0.6307, Val Loss: 0.6708, Train AUC-ROC: 0.6816, Val AUC-ROC: 0.7035, Train Accuracy: 0.6619, Val Accuracy: 0.6168
Epoch 4/100, Fold 1 - Train Loss: 0.6224, Val Loss: 0.6656, Train AUC-ROC: 0.6971, Val AUC-ROC: 0.7196, Train Accuracy: 0.6665, Val Accuracy: 0.6264
Epoch 5/100, Fold 1 - Train Loss: 0.6072, Val Loss: 0.6568, Train AUC-ROC: 0.7364, Val AUC-ROC: 0.7308, Train Accuracy: 0.6775, Val Accuracy: 0.6419
Epoch 6/100, Fold 1 - Train Loss: 0.6018, Val Loss: 0.6445, Train AUC-ROC: 0.7514, Val AUC-ROC: 0.7381, Train Accuracy: 0.7021, Val Accuracy: 0.6691
Epoch 7/100, Fold 1 - Train Loss: 0.5967, Val Loss: 0.6286, Train AUC-ROC: 0.7523, Val AUC-ROC: 0.7439, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 2 - Train Loss: 0.8072, Val Loss: 0.7101, Train AUC-ROC: 0.4868, Val AUC-ROC: 0.6397, Train Accuracy: 0.3866, Val Accuracy: 0.4105
Epoch 2/100, Fold 2 - Train Loss: 0.7865, Val Loss: 0.7142, Train AUC-ROC: 0.5688, Val AUC-ROC: 0.6994, Train Accuracy: 0.4353, Val Accuracy: 0.4178
Epoch 3/100, Fold 2 - Train Loss: 0.7673, Val Loss: 0.7199, Train AUC-ROC: 0.6317, Val AUC-ROC: 0.7296, Train Accuracy: 0.4637, Val Accuracy: 0.4119
Epoch 4/100, Fold 2 - Train Loss: 0.7528, Val Loss: 0.7239, Train AUC-ROC: 0.6749, Val AUC-ROC: 0.7438, Train Accuracy: 0.4956, Val Accuracy: 0.4053
Epoch 5/100, Fold 2 - Train Loss: 0.7406, Val Loss: 0.7240, Train AUC-ROC: 0.6993, Val AUC-ROC: 0.7510, Train Accuracy: 0.5063, Val Accuracy: 0.4245
Epoch 6/100, Fold 2 - Train Loss: 0.7350, Val Loss: 0.7186, Train AUC-ROC: 0.7196, Val AUC-ROC: 0.7566, Train Accuracy: 0.5203, Val Accuracy: 0.4591
Epoch 7/100, Fold 2 - Train Loss: 0.7209, Val Loss: 0.7056, Train AUC-ROC: 0.7492, Val AUC-ROC: 0.7622, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 1/100, Fold 8 - Train Loss: 0.7000, Val Loss: 0.6669, Train AUC-ROC: 0.4696, Val AUC-ROC: 0.5859, Train Accuracy: 0.5324, Val Accuracy: 0.6861
Epoch 2/100, Fold 8 - Train Loss: 0.6745, Val Loss: 0.6662, Train AUC-ROC: 0.5730, Val AUC-ROC: 0.6480, Train Accuracy: 0.5990, Val Accuracy: 0.6448
Epoch 3/100, Fold 8 - Train Loss: 0.6573, Val Loss: 0.6662, Train AUC-ROC: 0.6353, Val AUC-ROC: 0.6771, Train Accuracy: 0.6261, Val Accuracy: 0.6197
Epoch 4/100, Fold 8 - Train Loss: 0.6421, Val Loss: 0.6668, Train AUC-ROC: 0.6843, Val AUC-ROC: 0.6993, Train Accuracy: 0.6416, Val Accuracy: 0.6080
Epoch 5/100, Fold 8 - Train Loss: 0.6322, Val Loss: 0.6661, Train AUC-ROC: 0.7164, Val AUC-ROC: 0.7177, Train Accuracy: 0.6512, Val Accuracy: 0.6028
Epoch 6/100, Fold 8 - Train Loss: 0.6250, Val Loss: 0.6636, Train AUC-ROC: 0.7268, Val AUC-ROC: 0.7337, Train Accuracy: 0.6545, Val Accuracy: 0.6146
Epoch 7/100, Fold 8 - Train Loss: 0.6181, Val Loss: 0.6576, Train AUC-ROC: 0.7479, Val AUC-ROC: 0.7460, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 9 - Train Loss: 0.7579, Val Loss: 0.7161, Train AUC-ROC: 0.4703, Val AUC-ROC: 0.5724, Train Accuracy: 0.4244, Val Accuracy: 0.3014
Epoch 2/100, Fold 9 - Train Loss: 0.7380, Val Loss: 0.7142, Train AUC-ROC: 0.5558, Val AUC-ROC: 0.6461, Train Accuracy: 0.4770, Val Accuracy: 0.4075
Epoch 3/100, Fold 9 - Train Loss: 0.7162, Val Loss: 0.7120, Train AUC-ROC: 0.6326, Val AUC-ROC: 0.6814, Train Accuracy: 0.5400, Val Accuracy: 0.4407
Epoch 4/100, Fold 9 - Train Loss: 0.7033, Val Loss: 0.7099, Train AUC-ROC: 0.6746, Val AUC-ROC: 0.7069, Train Accuracy: 0.5660, Val Accuracy: 0.4532
Epoch 5/100, Fold 9 - Train Loss: 0.6908, Val Loss: 0.7072, Train AUC-ROC: 0.7077, Val AUC-ROC: 0.7270, Train Accuracy: 0.5819, Val Accuracy: 0.4731
Epoch 6/100, Fold 9 - Train Loss: 0.6870, Val Loss: 0.7037, Train AUC-ROC: 0.7138, Val AUC-ROC: 0.7439, Train Accuracy: 0.5942, Val Accuracy: 0.4893
Epoch 7/100, Fold 9 - Train Loss: 0.6687, Val Loss: 0.6977, Train AUC-ROC: 0.7561, Val AUC-ROC: 0.7565, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 10 - Train Loss: 0.7680, Val Loss: 0.6416, Train AUC-ROC: 0.5030, Val AUC-ROC: 0.6066, Train Accuracy: 0.4460, Val Accuracy: 0.7163
Epoch 2/100, Fold 10 - Train Loss: 0.7407, Val Loss: 0.6396, Train AUC-ROC: 0.5913, Val AUC-ROC: 0.6548, Train Accuracy: 0.4998, Val Accuracy: 0.6684
Epoch 3/100, Fold 10 - Train Loss: 0.7242, Val Loss: 0.6377, Train AUC-ROC: 0.6595, Val AUC-ROC: 0.6866, Train Accuracy: 0.5247, Val Accuracy: 0.6529
Epoch 4/100, Fold 10 - Train Loss: 0.7045, Val Loss: 0.6355, Train AUC-ROC: 0.7005, Val AUC-ROC: 0.7105, Train Accuracy: 0.5485, Val Accuracy: 0.6514
Epoch 5/100, Fold 10 - Train Loss: 0.6992, Val Loss: 0.6324, Train AUC-ROC: 0.7147, Val AUC-ROC: 0.7299, Train Accuracy: 0.5579, Val Accuracy: 0.6551
Epoch 6/100, Fold 10 - Train Loss: 0.6799, Val Loss: 0.6277, Train AUC-ROC: 0.7546, Val AUC-ROC: 0.7460, Train Accuracy: 0.5885, Val Accuracy: 0.6640
Epoch 7/100, Fold 10 - Train Loss: 0.6757, Val Loss: 0.6203, Train AUC-ROC: 0.7604, Val AUC-ROC: 0.7

# 6,12t

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (using 6H and 12H)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))

# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (using 6H and 12H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        six_hour = convert_to_one_hot(sample['6H'], self.total_variables)
        twelve_hour = convert_to_one_hot(sample['12H'], self.total_variables)
        return six_hour, twelve_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)  # dropout=0.0으로 설정
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = x[:, -1, :]
        #x = x.mean(dim=1)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer_6h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.transformer_12h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size * 2, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, six_hour, twelve_hour):
        transformer_out_6h = self.transformer_6h(six_hour)
        transformer_out_12h = self.transformer_12h(twelve_hour)
        transformer_out_combined = torch.cat((transformer_out_6h, transformer_out_12h), dim=1)
        output = self.mlp(transformer_out_combined)
        return output

# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for six_hour, twelve_hour, labels in val_loader:
            six_hour, twelve_hour, labels = six_hour.to(device), twelve_hour.to(device), labels.to(device)
            outputs = model(six_hour, twelve_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for six_hour, twelve_hour, labels in train_loader:
            six_hour, twelve_hour, labels = six_hour.to(device), twelve_hour.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(six_hour, twelve_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch
        }

        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    if best_model_state is not None:
        model_name = f'best_transformer_6H12H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel2/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

all_fold_results = []

start_time = time.time()

fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    all_fold_results.extend(fold_results)

    fold += 1

results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_6H12H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata2/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

end_time = time.time()
total_time = end_time - start_time
print(f"Time taken for 6H+12H training: {total_time:.2f} seconds")


/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 1 - Train Loss: 0.6663, Val Loss: 0.6525, Train AUC-ROC: 0.4840, Val AUC-ROC: 0.6713, Train Accuracy: 0.5859, Val Accuracy: 0.7074
Epoch 2/100, Fold 1 - Train Loss: 0.6336, Val Loss: 0.6410, Train AUC-ROC: 0.6169, Val AUC-ROC: 0.7425, Train Accuracy: 0.6493, Val Accuracy: 0.7097
Epoch 3/100, Fold 1 - Train Loss: 0.6046, Val Loss: 0.6276, Train AUC-ROC: 0.7258, Val AUC-ROC: 0.7713, Train Accuracy: 0.6940, Val Accuracy: 0.7148
Epoch 4/100, Fold 1 - Train Loss: 0.5846, Val Loss: 0.6105, Train AUC-ROC: 0.7745, Val AUC-ROC: 0.7872, Train Accuracy: 0.7209, Val Accuracy: 0.7325
Epoch 5/100, Fold 1 - Train Loss: 0.5645, Val Loss: 0.5899, Train AUC-ROC: 0.8123, Val AUC-ROC: 0.7997, Train Accuracy: 0.7353, Val Accuracy: 0.7590
Epoch 6/100, Fold 1 - Train Loss: 0.5527, Val Loss: 0.5665, Train AUC-ROC: 0.8353, Val AUC-ROC: 0.8097, Train Accuracy: 0.7465, Val Accuracy: 0.7841
Epoch 7/100, Fold 1 - Train Loss: 0.5383, Val Loss: 0.5409, Train AUC-ROC: 0.8586, Val AUC-ROC: 0.8180, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 2 - Train Loss: 0.7092, Val Loss: 0.6633, Train AUC-ROC: 0.5341, Val AUC-ROC: 0.6672, Train Accuracy: 0.5251, Val Accuracy: 0.6485
Epoch 2/100, Fold 2 - Train Loss: 0.6733, Val Loss: 0.6591, Train AUC-ROC: 0.6672, Val AUC-ROC: 0.7511, Train Accuracy: 0.5889, Val Accuracy: 0.6536
Epoch 3/100, Fold 2 - Train Loss: 0.6486, Val Loss: 0.6544, Train AUC-ROC: 0.7596, Val AUC-ROC: 0.7873, Train Accuracy: 0.6217, Val Accuracy: 0.6419
Epoch 4/100, Fold 2 - Train Loss: 0.6351, Val Loss: 0.6473, Train AUC-ROC: 0.7933, Val AUC-ROC: 0.8052, Train Accuracy: 0.6397, Val Accuracy: 0.6625
Epoch 5/100, Fold 2 - Train Loss: 0.6153, Val Loss: 0.6363, Train AUC-ROC: 0.8339, Val AUC-ROC: 0.8170, Train Accuracy: 0.6606, Val Accuracy: 0.6772
Epoch 6/100, Fold 2 - Train Loss: 0.5991, Val Loss: 0.6204, Train AUC-ROC: 0.8516, Val AUC-ROC: 0.8252, Train Accuracy: 0.6838, Val Accuracy: 0.7067
Epoch 7/100, Fold 2 - Train Loss: 0.5884, Val Loss: 0.5999, Train AUC-ROC: 0.8599, Val AUC-ROC: 0.8313, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 1/100, Fold 8 - Train Loss: 0.5593, Val Loss: 0.5411, Train AUC-ROC: 0.4813, Val AUC-ROC: 0.6281, Train Accuracy: 0.7607, Val Accuracy: 0.8246
Epoch 2/100, Fold 8 - Train Loss: 0.5217, Val Loss: 0.5277, Train AUC-ROC: 0.6186, Val AUC-ROC: 0.7490, Train Accuracy: 0.7907, Val Accuracy: 0.8261
Epoch 3/100, Fold 8 - Train Loss: 0.4928, Val Loss: 0.5167, Train AUC-ROC: 0.7212, Val AUC-ROC: 0.7967, Train Accuracy: 0.8025, Val Accuracy: 0.8460
Epoch 4/100, Fold 8 - Train Loss: 0.4691, Val Loss: 0.5064, Train AUC-ROC: 0.7821, Val AUC-ROC: 0.8170, Train Accuracy: 0.8254, Val Accuracy: 0.8629
Epoch 5/100, Fold 8 - Train Loss: 0.4507, Val Loss: 0.4955, Train AUC-ROC: 0.8269, Val AUC-ROC: 0.8274, Train Accuracy: 0.8422, Val Accuracy: 0.8703
Epoch 6/100, Fold 8 - Train Loss: 0.4370, Val Loss: 0.4828, Train AUC-ROC: 0.8510, Val AUC-ROC: 0.8346, Train Accuracy: 0.8414, Val Accuracy: 0.8674
Epoch 7/100, Fold 8 - Train Loss: 0.4304, Val Loss: 0.4677, Train AUC-ROC: 0.8595, Val AUC-ROC: 0.8397, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 9 - Train Loss: 0.6130, Val Loss: 0.6108, Train AUC-ROC: 0.4950, Val AUC-ROC: 0.6679, Train Accuracy: 0.6809, Val Accuracy: 0.7951
Epoch 2/100, Fold 9 - Train Loss: 0.5704, Val Loss: 0.5980, Train AUC-ROC: 0.6312, Val AUC-ROC: 0.7601, Train Accuracy: 0.7258, Val Accuracy: 0.7981
Epoch 3/100, Fold 9 - Train Loss: 0.5402, Val Loss: 0.5835, Train AUC-ROC: 0.7423, Val AUC-ROC: 0.8006, Train Accuracy: 0.7611, Val Accuracy: 0.8077
Epoch 4/100, Fold 9 - Train Loss: 0.5238, Val Loss: 0.5662, Train AUC-ROC: 0.7795, Val AUC-ROC: 0.8232, Train Accuracy: 0.7749, Val Accuracy: 0.8047
Epoch 5/100, Fold 9 - Train Loss: 0.5041, Val Loss: 0.5458, Train AUC-ROC: 0.8133, Val AUC-ROC: 0.8375, Train Accuracy: 0.7876, Val Accuracy: 0.8150
Epoch 6/100, Fold 9 - Train Loss: 0.4859, Val Loss: 0.5225, Train AUC-ROC: 0.8424, Val AUC-ROC: 0.8463, Train Accuracy: 0.8022, Val Accuracy: 0.8195
Epoch 7/100, Fold 9 - Train Loss: 0.4781, Val Loss: 0.4969, Train AUC-ROC: 0.8585, Val AUC-ROC: 0.8520, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 10 - Train Loss: 0.8434, Val Loss: 0.8107, Train AUC-ROC: 0.6120, Val AUC-ROC: 0.7548, Train Accuracy: 0.3525, Val Accuracy: 0.2174
Epoch 2/100, Fold 10 - Train Loss: 0.8103, Val Loss: 0.7917, Train AUC-ROC: 0.7058, Val AUC-ROC: 0.7961, Train Accuracy: 0.3947, Val Accuracy: 0.2660
Epoch 3/100, Fold 10 - Train Loss: 0.7778, Val Loss: 0.7679, Train AUC-ROC: 0.7756, Val AUC-ROC: 0.8181, Train Accuracy: 0.4489, Val Accuracy: 0.3478
Epoch 4/100, Fold 10 - Train Loss: 0.7605, Val Loss: 0.7394, Train AUC-ROC: 0.8064, Val AUC-ROC: 0.8332, Train Accuracy: 0.4871, Val Accuracy: 0.4370
Epoch 5/100, Fold 10 - Train Loss: 0.7424, Val Loss: 0.7063, Train AUC-ROC: 0.8256, Val AUC-ROC: 0.8439, Train Accuracy: 0.5221, Val Accuracy: 0.5151
Epoch 6/100, Fold 10 - Train Loss: 0.7234, Val Loss: 0.6706, Train AUC-ROC: 0.8475, Val AUC-ROC: 0.8513, Train Accuracy: 0.5500, Val Accuracy: 0.6116
Epoch 7/100, Fold 10 - Train Loss: 0.7091, Val Loss: 0.6340, Train AUC-ROC: 0.8541, Val AUC-ROC: 0.8

# 12,24

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (using 12H and 24H)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))

# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (using 12H and 24H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        twelve_hour = convert_to_one_hot(sample['12H'], self.total_variables)
        twenty_four_hour = convert_to_one_hot(sample['24H'], self.total_variables)
        return twelve_hour, twenty_four_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)  # dropout=0.0으로 설정
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = x[:, -1, :]
        #x = x.mean(dim=1)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer_12h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.transformer_24h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size * 2, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, twelve_hour, twenty_four_hour):
        transformer_out_12h = self.transformer_12h(twelve_hour)
        transformer_out_24h = self.transformer_24h(twenty_four_hour)
        transformer_out_combined = torch.cat((transformer_out_12h, transformer_out_24h), dim=1)
        output = self.mlp(transformer_out_combined)
        return output

# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for twelve_hour, twenty_four_hour, labels in val_loader:
            twelve_hour, twenty_four_hour, labels = twelve_hour.to(device), twenty_four_hour.to(device), labels.to(device)
            outputs = model(twelve_hour, twenty_four_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for twelve_hour, twenty_four_hour, labels in train_loader:
            twelve_hour, twenty_four_hour, labels = twelve_hour.to(device), twenty_four_hour.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(twelve_hour, twenty_four_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch
        }

        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    if best_model_state is not None:
        model_name = f'best_transformer_12H24H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel2(icd_10)/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

all_fold_results = []

start_time = time.time()

fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    all_fold_results.extend(fold_results)

    fold += 1

results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_12H24H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata2(icd_10)/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

end_time = time.time()
total_time = end_time - start_time
print(f"Time taken for 12H+24H training: {total_time:.2f} seconds")


/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 1 - Train Loss: 0.6160, Val Loss: 0.6543, Train AUC-ROC: 0.5202, Val AUC-ROC: 0.7200, Train Accuracy: 0.6669, Val Accuracy: 0.6566
Epoch 2/100, Fold 1 - Train Loss: 0.5748, Val Loss: 0.6455, Train AUC-ROC: 0.6645, Val AUC-ROC: 0.7754, Train Accuracy: 0.7163, Val Accuracy: 0.6647
Epoch 3/100, Fold 1 - Train Loss: 0.5489, Val Loss: 0.6327, Train AUC-ROC: 0.7459, Val AUC-ROC: 0.8033, Train Accuracy: 0.7435, Val Accuracy: 0.6735
Epoch 4/100, Fold 1 - Train Loss: 0.5280, Val Loss: 0.6152, Train AUC-ROC: 0.8010, Val AUC-ROC: 0.8192, Train Accuracy: 0.7666, Val Accuracy: 0.6949
Epoch 5/100, Fold 1 - Train Loss: 0.5149, Val Loss: 0.5919, Train AUC-ROC: 0.8246, Val AUC-ROC: 0.8298, Train Accuracy: 0.7729, Val Accuracy: 0.7281
Epoch 6/100, Fold 1 - Train Loss: 0.5038, Val Loss: 0.5633, Train AUC-ROC: 0.8475, Val AUC-ROC: 0.8375, Train Accuracy: 0.7801, Val Accuracy: 0.7590
Epoch 7/100, Fold 1 - Train Loss: 0.4858, Val Loss: 0.5308, Train AUC-ROC: 0.8691, Val AUC-ROC: 0.8431, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 2 - Train Loss: 0.6848, Val Loss: 0.6232, Train AUC-ROC: 0.5025, Val AUC-ROC: 0.7043, Train Accuracy: 0.5743, Val Accuracy: 0.7517
Epoch 2/100, Fold 2 - Train Loss: 0.6390, Val Loss: 0.6190, Train AUC-ROC: 0.6565, Val AUC-ROC: 0.7786, Train Accuracy: 0.6394, Val Accuracy: 0.7443
Epoch 3/100, Fold 2 - Train Loss: 0.6090, Val Loss: 0.6127, Train AUC-ROC: 0.7502, Val AUC-ROC: 0.8084, Train Accuracy: 0.6763, Val Accuracy: 0.7340
Epoch 4/100, Fold 2 - Train Loss: 0.5898, Val Loss: 0.6010, Train AUC-ROC: 0.7938, Val AUC-ROC: 0.8236, Train Accuracy: 0.7013, Val Accuracy: 0.7421
Epoch 5/100, Fold 2 - Train Loss: 0.5760, Val Loss: 0.5830, Train AUC-ROC: 0.8259, Val AUC-ROC: 0.8326, Train Accuracy: 0.7207, Val Accuracy: 0.7480
Epoch 6/100, Fold 2 - Train Loss: 0.5593, Val Loss: 0.5588, Train AUC-ROC: 0.8488, Val AUC-ROC: 0.8384, Train Accuracy: 0.7356, Val Accuracy: 0.7738
Epoch 7/100, Fold 2 - Train Loss: 0.5490, Val Loss: 0.5299, Train AUC-ROC: 0.8630, Val AUC-ROC: 0.8431, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 1/100, Fold 8 - Train Loss: 0.7517, Val Loss: 0.7337, Train AUC-ROC: 0.4596, Val AUC-ROC: 0.5885, Train Accuracy: 0.4628, Val Accuracy: 0.3515
Epoch 2/100, Fold 8 - Train Loss: 0.7114, Val Loss: 0.7312, Train AUC-ROC: 0.6046, Val AUC-ROC: 0.7112, Train Accuracy: 0.5149, Val Accuracy: 0.3906
Epoch 3/100, Fold 8 - Train Loss: 0.6767, Val Loss: 0.7253, Train AUC-ROC: 0.7148, Val AUC-ROC: 0.7744, Train Accuracy: 0.5688, Val Accuracy: 0.4223
Epoch 4/100, Fold 8 - Train Loss: 0.6651, Val Loss: 0.7144, Train AUC-ROC: 0.7604, Val AUC-ROC: 0.8052, Train Accuracy: 0.5892, Val Accuracy: 0.4539
Epoch 5/100, Fold 8 - Train Loss: 0.6414, Val Loss: 0.6969, Train AUC-ROC: 0.8036, Val AUC-ROC: 0.8221, Train Accuracy: 0.6174, Val Accuracy: 0.5063
Epoch 6/100, Fold 8 - Train Loss: 0.6230, Val Loss: 0.6719, Train AUC-ROC: 0.8359, Val AUC-ROC: 0.8318, Train Accuracy: 0.6429, Val Accuracy: 0.5696
Epoch 7/100, Fold 8 - Train Loss: 0.6107, Val Loss: 0.6397, Train AUC-ROC: 0.8510, Val AUC-ROC: 0.8385, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 9 - Train Loss: 0.6941, Val Loss: 0.7232, Train AUC-ROC: 0.5421, Val AUC-ROC: 0.7088, Train Accuracy: 0.5417, Val Accuracy: 0.3847
Epoch 2/100, Fold 9 - Train Loss: 0.6610, Val Loss: 0.7100, Train AUC-ROC: 0.6650, Val AUC-ROC: 0.7854, Train Accuracy: 0.5994, Val Accuracy: 0.4480
Epoch 3/100, Fold 9 - Train Loss: 0.6325, Val Loss: 0.6900, Train AUC-ROC: 0.7657, Val AUC-ROC: 0.8170, Train Accuracy: 0.6350, Val Accuracy: 0.5122
Epoch 4/100, Fold 9 - Train Loss: 0.6164, Val Loss: 0.6630, Train AUC-ROC: 0.7993, Val AUC-ROC: 0.8328, Train Accuracy: 0.6563, Val Accuracy: 0.5910
Epoch 5/100, Fold 9 - Train Loss: 0.6057, Val Loss: 0.6308, Train AUC-ROC: 0.8294, Val AUC-ROC: 0.8410, Train Accuracy: 0.6737, Val Accuracy: 0.6647
Epoch 6/100, Fold 9 - Train Loss: 0.5943, Val Loss: 0.5949, Train AUC-ROC: 0.8479, Val AUC-ROC: 0.8459, Train Accuracy: 0.6831, Val Accuracy: 0.7207
Epoch 7/100, Fold 9 - Train Loss: 0.5753, Val Loss: 0.5573, Train AUC-ROC: 0.8623, Val AUC-ROC: 0.8496, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 10 - Train Loss: 0.6356, Val Loss: 0.6381, Train AUC-ROC: 0.5830, Val AUC-ROC: 0.7640, Train Accuracy: 0.6433, Val Accuracy: 0.7170
Epoch 2/100, Fold 10 - Train Loss: 0.6045, Val Loss: 0.6283, Train AUC-ROC: 0.6848, Val AUC-ROC: 0.8167, Train Accuracy: 0.6810, Val Accuracy: 0.7170
Epoch 3/100, Fold 10 - Train Loss: 0.5754, Val Loss: 0.6120, Train AUC-ROC: 0.7742, Val AUC-ROC: 0.8409, Train Accuracy: 0.7163, Val Accuracy: 0.7288
Epoch 4/100, Fold 10 - Train Loss: 0.5585, Val Loss: 0.5888, Train AUC-ROC: 0.8089, Val AUC-ROC: 0.8541, Train Accuracy: 0.7297, Val Accuracy: 0.7517
Epoch 5/100, Fold 10 - Train Loss: 0.5466, Val Loss: 0.5599, Train AUC-ROC: 0.8308, Val AUC-ROC: 0.8640, Train Accuracy: 0.7435, Val Accuracy: 0.7833
Epoch 12/100, Fold 10 - Train Loss: 0.4703, Val Loss: 0.4145, Train AUC-ROC: 0.9076, Val AUC-ROC: 0.8845, Train Accuracy: 0.8389, Val Accuracy: 0.8939
Epoch 13/100, Fold 10 - Train Loss: 0.4682, Val Loss: 0.4163, Train AUC-ROC: 0.9099, Val AUC-ROC: 0

# 6,12,24t

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (only using 6H, 12H, and 24H)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))

# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (using 6H, 12H, and 24H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        six_hour = convert_to_one_hot(sample['6H'], self.total_variables)
        twelve_hour = convert_to_one_hot(sample['12H'], self.total_variables)
        twenty_four_hour = convert_to_one_hot(sample['24H'], self.total_variables)
        return six_hour, twelve_hour, twenty_four_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        #x = x[:, -1, :]
        x = x.mean(dim=1)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer_6h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.transformer_12h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.transformer_24h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size * 3, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, six_hour, twelve_hour, twenty_four_hour):
        transformer_out_6h = self.transformer_6h(six_hour)
        transformer_out_12h = self.transformer_12h(twelve_hour)
        transformer_out_24h = self.transformer_24h(twenty_four_hour)
        combined_out = torch.cat((transformer_out_6h, transformer_out_12h, transformer_out_24h), dim=1)
        output = self.mlp(combined_out)
        return output

# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for six_hour, twelve_hour, twenty_four_hour, labels in val_loader:
            six_hour, twelve_hour, twenty_four_hour, labels = six_hour.to(device), twelve_hour.to(device), twenty_four_hour.to(device), labels.to(device)
            outputs = model(six_hour, twelve_hour, twenty_four_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []  # To store metrics for each epoch

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for six_hour, twelve_hour, twenty_four_hour, labels in train_loader:
            six_hour, twelve_hour, twenty_four_hour, labels = six_hour.to(device), twelve_hour.to(device), twenty_four_hour.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(six_hour, twelve_hour, twenty_four_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        # Log the metrics for this epoch
        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,  # Save current fold number
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch  # Track the best epoch
        }

        # Add this epoch's result to the list
        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    # Save the best model
    if best_model_state is not None:
        model_name = f'best_transformer_6H_12H_24H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel3(icd_10)/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Set the seed for reproducibility
# seed = 42  # You can choose any seed value
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize lists to store results for all folds
all_fold_results = []

# Start tracking time for the 6H, 12H, and 24H training
start_time = time.time()

# Loop through the folds
fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    # Prepare data loaders for this fold (using 6H, 12H, and 24H data)
    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    # Initialize model
    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train the model and capture results for this fold
    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    # Append the results for this fold to all_fold_results
    all_fold_results.extend(fold_results)

    fold += 1

# After all folds are done, save the results to a single CSV file
results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_6H_12H_24H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata3(icd_10)/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

# Stop the timer for the 6H, 12H, and 24H training
end_time = time.time()
total_time = end_time - start_time

# Display the total time taken for the training
print(f"Time taken for 6H, 12H, and 24H training: {total_time:.2f} seconds")


/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 1 - Train Loss: 0.8944, Val Loss: 0.8852, Train AUC-ROC: 0.4335, Val AUC-ROC: 0.6715, Train Accuracy: 0.3027, Val Accuracy: 0.1776
Epoch 2/100, Fold 1 - Train Loss: 0.8405, Val Loss: 0.8819, Train AUC-ROC: 0.6170, Val AUC-ROC: 0.7571, Train Accuracy: 0.3608, Val Accuracy: 0.1828
Epoch 3/100, Fold 1 - Train Loss: 0.8085, Val Loss: 0.8722, Train AUC-ROC: 0.7320, Val AUC-ROC: 0.7882, Train Accuracy: 0.4253, Val Accuracy: 0.2137
Epoch 4/100, Fold 1 - Train Loss: 0.7885, Val Loss: 0.8551, Train AUC-ROC: 0.7741, Val AUC-ROC: 0.8049, Train Accuracy: 0.4688, Val Accuracy: 0.2520
Epoch 5/100, Fold 1 - Train Loss: 0.7712, Val Loss: 0.8308, Train AUC-ROC: 0.8099, Val AUC-ROC: 0.8165, Train Accuracy: 0.5026, Val Accuracy: 0.3058
Epoch 6/100, Fold 1 - Train Loss: 0.7537, Val Loss: 0.7974, Train AUC-ROC: 0.8295, Val AUC-ROC: 0.8255, Train Accuracy: 0.5225, Val Accuracy: 0.3736
Epoch 7/100, Fold 1 - Train Loss: 0.7394, Val Loss: 0.7571, Train AUC-ROC: 0.8374, Val AUC-ROC: 0.8338, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 2 - Train Loss: 0.6853, Val Loss: 0.6769, Train AUC-ROC: 0.5364, Val AUC-ROC: 0.7569, Train Accuracy: 0.5673, Val Accuracy: 0.5711
Epoch 2/100, Fold 2 - Train Loss: 0.6383, Val Loss: 0.6709, Train AUC-ROC: 0.7034, Val AUC-ROC: 0.7956, Train Accuracy: 0.6324, Val Accuracy: 0.5689
Epoch 3/100, Fold 2 - Train Loss: 0.6102, Val Loss: 0.6610, Train AUC-ROC: 0.7709, Val AUC-ROC: 0.8112, Train Accuracy: 0.6659, Val Accuracy: 0.5932
Epoch 4/100, Fold 2 - Train Loss: 0.5988, Val Loss: 0.6454, Train AUC-ROC: 0.8048, Val AUC-ROC: 0.8200, Train Accuracy: 0.6722, Val Accuracy: 0.6271
Epoch 5/100, Fold 2 - Train Loss: 0.5805, Val Loss: 0.6235, Train AUC-ROC: 0.8349, Val AUC-ROC: 0.8247, Train Accuracy: 0.7002, Val Accuracy: 0.6750
Epoch 6/100, Fold 2 - Train Loss: 0.5665, Val Loss: 0.5956, Train AUC-ROC: 0.8514, Val AUC-ROC: 0.8285, Train Accuracy: 0.7229, Val Accuracy: 0.7229
Epoch 7/100, Fold 2 - Train Loss: 0.5550, Val Loss: 0.5644, Train AUC-ROC: 0.8640, Val AUC-ROC: 0.8316, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

Epoch 1/100, Fold 5 - Train Loss: 0.5539, Val Loss: 0.5760, Train AUC-ROC: 0.4928, Val AUC-ROC: 0.6902, Train Accuracy: 0.7860, Val Accuracy: 0.8268
Epoch 2/100, Fold 5 - Train Loss: 0.5108, Val Loss: 0.5713, Train AUC-ROC: 0.6658, Val AUC-ROC: 0.7653, Train Accuracy: 0.8053, Val Accuracy: 0.8371
Epoch 3/100, Fold 5 - Train Loss: 0.4887, Val Loss: 0.5667, Train AUC-ROC: 0.7435, Val AUC-ROC: 0.7952, Train Accuracy: 0.8182, Val Accuracy: 0.8349
Epoch 4/100, Fold 5 - Train Loss: 0.4705, Val Loss: 0.5611, Train AUC-ROC: 0.7924, Val AUC-ROC: 0.8111, Train Accuracy: 0.8225, Val Accuracy: 0.8172
Epoch 5/100, Fold 5 - Train Loss: 0.4589, Val Loss: 0.5528, Train AUC-ROC: 0.8226, Val AUC-ROC: 0.8230, Train Accuracy: 0.8276, Val Accuracy: 0.8121
Epoch 6/100, Fold 5 - Train Loss: 0.4525, Val Loss: 0.5410, Train AUC-ROC: 0.8359, Val AUC-ROC: 0.8324, Train Accuracy: 0.8330, Val Accuracy: 0.8091
Epoch 7/100, Fold 5 - Train Loss: 0.4430, Val Loss: 0.5255, Train AUC-ROC: 0.8531, Val AUC-ROC: 0.8398, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 6 - Train Loss: 0.7525, Val Loss: 0.6702, Train AUC-ROC: 0.4833, Val AUC-ROC: 0.6949, Train Accuracy: 0.4537, Val Accuracy: 0.6227
Epoch 2/100, Fold 6 - Train Loss: 0.7168, Val Loss: 0.6770, Train AUC-ROC: 0.6305, Val AUC-ROC: 0.7663, Train Accuracy: 0.5206, Val Accuracy: 0.5866
Epoch 3/100, Fold 6 - Train Loss: 0.6814, Val Loss: 0.6815, Train AUC-ROC: 0.7308, Val AUC-ROC: 0.7991, Train Accuracy: 0.5914, Val Accuracy: 0.5704
Epoch 4/100, Fold 6 - Train Loss: 0.6655, Val Loss: 0.6808, Train AUC-ROC: 0.7772, Val AUC-ROC: 0.8191, Train Accuracy: 0.6075, Val Accuracy: 0.5903
Epoch 5/100, Fold 6 - Train Loss: 0.6532, Val Loss: 0.6728, Train AUC-ROC: 0.8004, Val AUC-ROC: 0.8327, Train Accuracy: 0.6315, Val Accuracy: 0.6021
Epoch 6/100, Fold 6 - Train Loss: 0.6323, Val Loss: 0.6571, Train AUC-ROC: 0.8286, Val AUC-ROC: 0.8418, Train Accuracy: 0.6532, Val Accuracy: 0.6389
Epoch 7/100, Fold 6 - Train Loss: 0.6231, Val Loss: 0.6345, Train AUC-ROC: 0.8381, Val AUC-ROC: 0.8477, Tr

/home/jhy0812/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Fold 7 - Train Loss: 0.8309, Val Loss: 0.7748, Train AUC-ROC: 0.4686, Val AUC-ROC: 0.6501, Train Accuracy: 0.3822, Val Accuracy: 0.2410
Epoch 2/100, Fold 7 - Train Loss: 0.7751, Val Loss: 0.7776, Train AUC-ROC: 0.6396, Val AUC-ROC: 0.7313, Train Accuracy: 0.4613, Val Accuracy: 0.2933
Epoch 3/100, Fold 7 - Train Loss: 0.7434, Val Loss: 0.7753, Train AUC-ROC: 0.7361, Val AUC-ROC: 0.7633, Train Accuracy: 0.5144, Val Accuracy: 0.3360
Epoch 4/100, Fold 7 - Train Loss: 0.7303, Val Loss: 0.7656, Train AUC-ROC: 0.7804, Val AUC-ROC: 0.7789, Train Accuracy: 0.5354, Val Accuracy: 0.3839
Epoch 5/100, Fold 7 - Train Loss: 0.7033, Val Loss: 0.7465, Train AUC-ROC: 0.8282, Val AUC-ROC: 0.7876, Train Accuracy: 0.5710, Val Accuracy: 0.4407
Epoch 6/100, Fold 7 - Train Loss: 0.6953, Val Loss: 0.7183, Train AUC-ROC: 0.8360, Val AUC-ROC: 0.7938, Train Accuracy: 0.5826, Val Accuracy: 0.5092
Epoch 7/100, Fold 7 - Train Loss: 0.6757, Val Loss: 0.6829, Train AUC-ROC: 0.8553, Val AUC-ROC: 0.7978, Tr

# 6,12,24,48t

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load and preprocess data
df = pd.read_csv('/data2/project/2024summer/jhy0812/prodata3(icd_10)/data_label_merge.tsv', sep='\t')

def parse_time_series(data):
    time_series = [item.split(',') for item in data.split('|')]
    return time_series

# Parse all time interval data
intervals = ['6H', '12H', '24H', '48H']
for interval in intervals:
    df[interval] = df[interval].apply(parse_time_series)

# Collect unique variable values from all intervals
unique_items = set()
for column in intervals:
    for series in df[column]:
        for item_list in series:
            filtered_items = [item for item in item_list if item.strip() != '']
            if filtered_items:
                unique_items.update(filtered_items)

# Create item-to-index mapping for all intervals
item_to_index = {item: idx for idx, item in enumerate(sorted(unique_items, key=lambda x: int(x)))}
total_variables = len(item_to_index)

# Convert variables to index for all intervals (using 6H, 12H, 24H, and 48H)
def convert_to_index(data, item_to_index):
    indexed_data = []
    for item_list in data:
        if item_list == ['']:
            indexed_data.append([])
        else:
            indexed_data.append([item_to_index[item] for item in item_list if item != ''])
    return indexed_data

for interval in intervals:
    df[interval] = df[interval].apply(lambda x: convert_to_index(x, item_to_index))

# Extract labels
labels = df['LABEL_DEAD1_ALIVE0'].values

# Dataset class (using 6H, 12H, 24H, and 48H data for training)
class EHRDataset(Dataset):
    def __init__(self, df, total_variables):
        self.df = df
        self.total_variables = total_variables

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        label = sample['LABEL_DEAD1_ALIVE0']
        six_hour = convert_to_one_hot(sample['6H'], self.total_variables)
        twelve_hour = convert_to_one_hot(sample['12H'], self.total_variables)
        twenty_four_hour = convert_to_one_hot(sample['24H'], self.total_variables)
        forty_eight_hour = convert_to_one_hot(sample['48H'], self.total_variables)
        return six_hour, twelve_hour, twenty_four_hour, forty_eight_hour, torch.tensor(label, dtype=torch.float32)

def convert_to_one_hot(data, total_variables):
    one_hot_vectors = []
    for indices in data:
        vector = torch.zeros(total_variables, dtype=torch.float32)
        for index in indices:
            if index < total_variables:
                vector[index] = 1.0
        one_hot_vectors.append(vector)
    return torch.stack(one_hot_vectors)

# Transformer Encoder Model without Dropout
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(TransformerEncoderModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dropout=0.0)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = x[:, -1, :]
        #x = x.mean(dim=1)
        return x

# Survival Prediction Model with Dropout in MLP only
class SurvivalPredictionModel(nn.Module):
    def __init__(self, transformer_hidden_size, mlp_hidden_size, num_heads, num_layers, dropout_prob):
        super(SurvivalPredictionModel, self).__init__()
        self.transformer_6h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.transformer_12h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.transformer_24h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.transformer_48h = TransformerEncoderModel(total_variables, transformer_hidden_size, num_heads, num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(transformer_hidden_size * 4, mlp_hidden_size),
            nn.BatchNorm1d(mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(mlp_hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, six_hour, twelve_hour, twenty_four_hour, forty_eight_hour):
        transformer_out_6h = self.transformer_6h(six_hour)
        transformer_out_12h = self.transformer_12h(twelve_hour)
        transformer_out_24h = self.transformer_24h(twenty_four_hour)
        transformer_out_48h = self.transformer_48h(forty_eight_hour)
        combined_out = torch.cat((transformer_out_6h, transformer_out_12h, transformer_out_24h, transformer_out_48h), dim=1)
        output = self.mlp(combined_out)
        return output

# Model evaluation function
def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []
    with torch.no_grad():
        for six_hour, twelve_hour, twenty_four_hour, forty_eight_hour, labels in val_loader:
            six_hour, twelve_hour, twenty_four_hour, forty_eight_hour, labels = (
                six_hour.to(device), twelve_hour.to(device), twenty_four_hour.to(device), forty_eight_hour.to(device), labels.to(device)
            )
            outputs = model(six_hour, twelve_hour, twenty_four_hour, forty_eight_hour).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend(outputs.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = accuracy_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_precision = precision_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_recall = recall_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_f1 = f1_score(val_labels, (np.array(val_preds) > 0.5).astype(int))
    val_auroc = roc_auc_score(val_labels, val_preds)

    return val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc

# Model training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10, fold=None):
    model.train()
    best_auc = 0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    results = []  # To store metrics for each epoch

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        train_loss = 0
        for six_hour, twelve_hour, twenty_four_hour, forty_eight_hour, labels in train_loader:
            six_hour, twelve_hour, twenty_four_hour, forty_eight_hour, labels = (
                six_hour.to(device), twelve_hour.to(device), twenty_four_hour.to(device), forty_eight_hour.to(device), labels.to(device)
            )
            optimizer.zero_grad()
            outputs = model(six_hour, twelve_hour, twenty_four_hour, forty_eight_hour).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_precision = precision_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_recall = recall_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_f1 = f1_score(train_labels, (np.array(train_preds) > 0.5).astype(int))
        train_auroc = roc_auc_score(train_labels, train_preds)

        val_loss, val_acc, val_precision, val_recall, val_f1, val_auroc = evaluate_model(model, val_loader, criterion)

        # Log the metrics for this epoch
        epoch_result = {
            'epoch': epoch + 1,
            'fold': fold,  # Save current fold number
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_f1': train_f1,
            'val_f1': val_f1,
            'train_auroc': train_auroc,
            'val_auroc': val_auroc,
            'best_epoch': best_epoch  # Track the best epoch
        }

        # Add this epoch's result to the list
        results.append(epoch_result)

        print(f"Epoch {epoch+1}/{num_epochs}, Fold {fold} - "
              f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train AUC-ROC: {train_auroc:.4f}, Val AUC-ROC: {val_auroc:.4f}, "
              f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

        if val_auroc > best_auc:
            best_auc = val_auroc
            best_epoch = epoch + 1
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()

    # Save the best model
    if best_model_state is not None:
        model_name = f'best_transformer_6H_12H_24H_48H_fold_{fold}.pt'
        torch.save(best_model_state, f'/data2/project/2024summer/jhy0812/transmodel3(icd_10)/{model_name}')
        print(f"Best model saved with AUROC: {best_auc:.4f} at epoch {best_epoch}")

    return results

# Set the seed for reproducibility
# seed = 42  # You can choose any seed value
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

# Cross-validation and training loop
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Initialize lists to store results for all folds
all_fold_results = []

# Start tracking time for the 6H, 12H, 24H, and 48H training
start_time = time.time()

# Loop through the folds
fold = 1
for train_index, val_index in sss.split(df, labels):
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    # Prepare data loaders for this fold (using 6H, 12H, 24H, and 48H data)
    train_dataset = EHRDataset(train_df, total_variables)
    val_dataset = EHRDataset(val_df, total_variables)

    train_loader = DataLoader(train_dataset, batch_size=8192, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8192, shuffle=False)

    # Initialize model
    model = SurvivalPredictionModel(
        transformer_hidden_size=128,
        mlp_hidden_size=64,
        num_heads=8,
        num_layers=2,
        dropout_prob=0.5
    ).to(device)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Train the model and capture results for this fold
    fold_results = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=20, fold=fold)

    # Append the results for this fold to all_fold_results
    all_fold_results.extend(fold_results)

    fold += 1

# After all folds are done, save the results to a single CSV file
results_df = pd.DataFrame(all_fold_results)
results_csv_name = 'transformer_6H_12H_24H_48H.csv'
results_df.to_csv(f'/data2/project/2024summer/jhy0812/transdata3(icd_10)/{results_csv_name}', index=False)
print(f"Saved results to {results_csv_name}")

# Stop the timer for the 6H, 12H, 24H, and 48H training
end_time = time.time()
total_time = end_time - start_time

# Display the total time taken for the training
print(f"Time taken for 6H, 12H, 24H, and 48H training: {total_time:.2f} seconds")
